<a href="https://colab.research.google.com/github/thanga-v2/DuckDb_Hands-on/blob/main/DuckDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DuckDB is RDMS that supports SQL.

Its ability to run SQL quries over pandas wihtout the need of importing or duplicating the data.

DuckDb is built with vectorized data processing.

MySql / Sqlite is row based storage.

Duckdb is columnar storage (column based)

In simple leveraging he strength of SQL without having a need of seperate SQL env

In columnar storage format, data gets stored column b y column rather than row by row in traditional (RDMS)

By reading only a specific column form the database, duckdb heps reduces the computation power required to load all the data.

By reading only the necessary columns from disk, DuckDB
significantly reduces the amount of data that needs to be
transferred into memory, speeding up query execution.


Vectorized Engine. 🇰

Duck db process the data in vectors.

operating with chuks of rows rather than processing the entire row.

Duck db designed to work on in-memory data structures. i.e it doesn't need to create copies which could potentially slow down the systems.

Ex If we are proforing aggragation, duck db can break the data into multiple chunks and allows them to process parallely into multiplt CPU cores.


Tuple at a time execution vs Columns at a time execution



In [1]:
!pip install duckdb

In [7]:
import duckdb

# creating a connection

persistant_conn = duckdb.connect("my_in_mem.db")

In [8]:
# creating an in-memory copy of the above created database

in_memory_conn = duckdb.connect(database = ":memory:")

whatever the changes we make in the in-memory will be lost once we shut down the system.

to retrang between the session we should use the persistent database which we have created above.


In [9]:
persistant_conn

In [10]:
# creating a table

persistant_conn.execute (
    '''
    create table virtuAI(
      employee_id INTEGER PRIMARY KEY,
      name VARCHAR,
      department VARCHAR,
      age INTEGER,
      role VARCHAR,
    )
    '''
)

In [11]:
persistant_conn.execute('SHOW TABLES').fetchall()

[('virtuAI',)]

In [14]:
# converting it into a data frame using df

persistant_conn.execute('SHOW TABLES').df()

,name
0,virtuAI


In [28]:
# insert

persistant_conn.execute(
    '''
    INSERT INTO virtuAI (employee_id, name, department, age, role)
    VALUES
    (1, 'thanga','AI & Blockchain', 30, 'PM'),
    (2, 'nawin', 'AI & Blockchain', 23, 'PM'),
    (3, 'kiruba', 'AI & Blockchain', 23, 'PM'),
    (4, 'viswa', 'AI', 22, 'AI engineering'),
    (5, 'praveen', 'Full stack', 23, 'Engineering'),
    (6, 'murali' , 'Full stack', 22, 'Engineering'),
    (7, 'prabhu', 'Full stack' , 23, 'Engineering'),
    '''
)

ConstraintException: Constraint Error: Duplicate key "employee_id: 1" violates primary key constraint. If this is an unexpected constraint violation please double check with the known index limitations section in our documentation (https://duckdb.org/docs/sql/indexes).

In [20]:
persistant_conn.execute('SELECT * FROM virtuAI')

In [21]:
persistant_conn.execute('SELECT * FROM virtuAI').fetchall()

[(1, 'thanga', 'AI & Blockchain', 30, 'PM'),
 (2, 'nawin', 'AI & Blockchain', 23, 'PM'),
 (3, 'kiruba', 'AI & Blockchain', 23, 'PM'),
 (4, 'viswa', 'AI', 22, 'PM'),
 (5, 'praveen', 'Full stack', 23, 'Engineering'),
 (6, 'murali', 'Full stack', 22, 'Engineering'),
 (7, 'prabhu', 'Full stack', 23, 'Engineering')]

In [22]:
persistant_conn.execute('SELECT * FROM virtuAI').df()

,employee_id,name,department,age,role
0,1,thanga,AI & Blockchain,30,PM
1,2,nawin,AI & Blockchain,23,PM
2,3,kiruba,AI & Blockchain,23,PM
3,4,viswa,AI,22,PM
4,5,praveen,Full stack,23,Engineering
5,6,murali,Full stack,22,Engineering
6,7,prabhu,Full stack,23,Engineering


In [27]:
persistant_conn.execute('SELECT * FROM virtuAI WHERE age < 25').df()

,employee_id,name,department,age,role
0,2,nawin,AI & Blockchain,23,PM
1,3,kiruba,AI & Blockchain,23,PM
2,4,viswa,AI,22,PM
3,5,praveen,Full stack,23,Engineering
4,6,murali,Full stack,22,Engineering
5,7,prabhu,Full stack,23,Engineering


In [29]:
persistant_conn.execute(
    '''
  SELECT department, COUNT(*) as employee_count
  FROM virtuAI
  GROUP BY department
  '''
).df()

,department,employee_count
0,Full stack,3
1,AI & Blockchain,3
2,AI,1


In [30]:
persistant_conn.execute(
    '''
  SELECT department, AVG(age) as average_age
  FROM virtuAI
  GROUP BY department
  '''
).df()

,department,average_age
0,Full stack,22.666667
1,AI & Blockchain,25.333333
2,AI,22.000000


In [32]:
persistant_conn.execute(
    '''
  SELECT department, AVG(age) as average_age
  FROM virtuAI
  GROUP BY department
  '''
).df()

,department,average_age
0,Full stack,22.666667
1,AI & Blockchain,25.333333
2,AI,22.000000


In [37]:
persistant_conn.execute(
    '''
    SELECT department, MAX(age) as senior_age
    FROM virtuAI
    GROUP BY department
    '''
).df()

,department,senior_age
0,Full stack,23
1,AI & Blockchain,30
2,AI,22


using pandas


    (1, 'thanga','AI & Blockchain', 30, 'PM'),
    (2, 'nawin', 'AI & Blockchain', 23, 'PM'),
    (3, 'kiruba', 'AI & Blockchain', 23, 'PM'),
    (4, 'viswa', 'AI', 22, 'AI engineering'),
    (5, 'praveen', 'Full stack', 23, 'Engineering'),
    (6, 'murali' , 'Full stack', 22, 'Engineering'),
    (7, 'prabhu', 'Full stack' , 23, 'Engineering'),

In [2]:
import pandas as pd

In [4]:
emp = pd.DataFrame({
    'employee_id' : [1,2,3,4,5,6,7],
    'name' : ['thanga','nawin', 'kiruba', 'viswa', 'praveen', 'murali', 'prabhu'],
    'department' : ['AI & Blockchain', 'AI & Blockchain', 'AI & Blockchain', 'AI', 'Full stack', 'Full stack', 'Full stack'],
    'age' : [30, 23, 23, 22, 23, 22, 23],
    'role' : ['PM', 'PM', 'PM', 'AI engineering', 'Engineering', 'Engineering', 'Engineering']
})

In [5]:
sales = pd.DataFrame({
    'employee_id' : [1,2,3,4,5,6,7],
    'sales_id': [102,109,11134,12,6,78,9]})

In [6]:
display(emp)

,employee_id,name,department,age,role
0,1,thanga,AI & Blockchain,30,PM
1,2,nawin,AI & Blockchain,23,PM
2,3,kiruba,AI & Blockchain,23,PM
3,4,viswa,AI,22,AI engineering
4,5,praveen,Full stack,23,Engineering
5,6,murali,Full stack,22,Engineering
6,7,prabhu,Full stack,23,Engineering


In [7]:
display(sales)

,employee_id,sales_id
0,1,102
1,2,109
2,3,11134
3,4,12
4,5,6
5,6,78
6,7,9


In [9]:
# Loading data from CSV

import duckdb

persistant_conn = duckdb.connect("my_in_mem.db")

In [11]:
query = '''
SELECT * from read_csv_auto('flights.csv')
'''

In [13]:
persistant_conn.execute(query).df()

,FlightDate,UniqueCarrier,OriginCityName,DestCityName
0,1988-01-01,AA,"New York, NY","Los Angeles, CA"
1,1988-01-02,AA,"New York, NY","Los Angeles, CA"
2,1988-01-03,AA,"New York, NY","Los Angeles, CA"


In [14]:
query = '''
Create table flights as
SELECT * from read_csv_auto('flights.csv')
'''

In [15]:
persistant_conn.execute(query).df()

,Count
0,3


In [16]:
# working with parquet files

query = '''
SELECT * from read_parquet('userdata1.parquet')
'''

In [20]:
persistant_conn.execute(query).df()

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 07:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,2016-02-03 17:04:03,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,
2,2016-02-03 01:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
3,2016-02-03 00:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,
4,2016-02-03 05:05:31,5,Carlos,Burns,cburns4@miitbeian.gov.cn,,169.113.235.40,5602256255204850,South Africa,,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2016-02-03 10:30:59,996,Dennis,Harris,dharrisrn@eepurl.com,Male,178.180.111.236,374288806662929,Greece,7/8/1965,263399.54,Editor,
996,2016-02-03 17:16:53,997,Gloria,Hamilton,ghamiltonro@rambler.ru,Female,71.50.39.137,,China,4/22/1975,83183.54,VP Product Management,
997,2016-02-03 05:02:20,998,Nancy,Morris,nmorrisrp@ask.com,,6.188.121.221,3553564071014997,Sweden,5/1/1979,NaN,Junior Executive,
998,2016-02-03 02:41:32,999,Annie,Daniels,adanielsrq@squidoo.com,Female,97.221.132.35,30424803513734,China,10/9/1991,18433.85,Editor,​


Query remote datasets

In [29]:
persistant_conn.close()

In [31]:
persistant_conn = duckdb.connect("my_in_mem.db")

In [38]:
query = '''
SELECT * from read_csv_auto('https://raw.githubusercontent.com/weimenglee/DuckDB_Book/main/AMZN.csv')
'''
result = persistant_conn.execute(query).df()
print(result)

           Date         Open         High          Low        Close  \
0    1997-05-15     2.437500     2.500000     1.927083     1.958333   
1    1997-05-16     1.968750     1.979167     1.708333     1.729167   
2    1997-05-19     1.760417     1.770833     1.625000     1.708333   
3    1997-05-20     1.729167     1.750000     1.635417     1.635417   
4    1997-05-21     1.635417     1.645833     1.375000     1.427083   
...         ...          ...          ...          ...          ...   
5660 2019-11-11  1778.000000  1780.000000  1767.130005  1771.650024   
5661 2019-11-12  1774.660034  1786.219971  1771.910034  1778.000000   
5662 2019-11-13  1773.390015  1775.000000  1747.319946  1753.109985   
5663 2019-11-14  1751.430054  1766.589966  1749.560059  1754.599976   
5664 2019-11-15  1760.050049  1761.680054  1732.859985  1739.489990   

        Adj Close    Volume  
0        1.958333  72156000  
1        1.729167  14700000  
2        1.708333   6106800  
3        1.635417   5467200

In [46]:
query = '''
SELECT * from read_csv_auto('https://raw.githubusercontent.com/weimenglee/DuckDB_Book/main/AMZN.csv') WHERE Year(Date) = 2018
'''

In [47]:
persistant_conn.execute(query).df()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-01-02,1172.000000,1190.000000,1170.510010,1189.010010,1189.010010,2694500
1,2018-01-03,1188.300049,1205.489990,1188.300049,1204.199951,1204.199951,3108800
2,2018-01-04,1205.000000,1215.869995,1204.660034,1209.589966,1209.589966,3022100
3,2018-01-05,1217.510010,1229.140015,1210.000000,1229.140015,1229.140015,3544700
4,2018-01-08,1236.000000,1253.079956,1232.030029,1246.869995,1246.869995,4279500
...,...,...,...,...,...,...,...
246,2018-12-24,1346.000000,1396.030029,1307.000000,1343.959961,1343.959961,7220000
247,2018-12-26,1368.890015,1473.160034,1363.010010,1470.900024,1470.900024,10411800
248,2018-12-27,1454.199951,1469.000000,1390.310059,1461.640015,1461.640015,9722000
249,2018-12-28,1473.349976,1513.469971,1449.000000,1478.020020,1478.020020,8829000
